### Assignment for Classification of Street View House Number Images ###
*Step 1 below is to access the folders on Google Drive where the images are stored* 

In [0]:
import tensorflow as tf

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd '/content/gdrive/My Drive'

/content/gdrive/My Drive


In [0]:
ls

 1540813978193..jpg  'CostaRica Share Cert copy -2.jpg'   sample_submission.csv
'CNN R7 Day 2'/      'NN R6 Day 1'/                       SVHN_single_grey1.h5
'Colab Notebooks'/   'NN R7 Day 1'/
